In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os
import tabula
import re
import PyPDF2
import locale
import io
import glob
import sys
import gc


os.environ['w2n.lang'] = 'es'
locale.setlocale(locale.LC_TIME, 'es_ES')

def fill_text_field(driver, xpath, text):
    # Funcion que llena texto en un campo de la pagina
    text_input = driver.find_element(By.XPATH, xpath)
    text_input.clear()
    text_input.send_keys(text)

def select_option(driver, xpath, index):
    # Selecciona una opcion en un menu desplegable
    select_object = Select(driver.find_element(By.XPATH, xpath))
    select_object.select_by_value(index)


url = 'https://www.pjud.cl/tribunales/corte-suprema'
chromeOptions = webdriver.ChromeOptions()
path = os.path.join(os.getcwd(), "output2\\")
prefs = {"download.default_directory" : path,  "directory_upgrade": True}
chromeOptions.add_experimental_option("prefs",prefs)
chromeOptions.add_argument("--headless")



driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options = chromeOptions)
driver.get(url)
#NOTA: al abrirse el driver de Chrome, seleccionar Configuración-Descargar PDFs

driver.implicitly_wait(10)

años = [2022]

meses = ["01", "02", "03", "04", "05", "06", "07", "08"]

for año in años:
    # Estos indices se obtienen de las opciones de la pagina, cada uno es un mes, cambiar para obtener mas meses
    #for index in range(1, 2):
        # Se envia la info a la pagina
    select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[1]/div/select', str(año))
    for mes in meses:
        select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[2]/div/select', mes)
        select = Select(driver.find_element(By.XPATH, "/html/body/div[11]/div/div/div[2]/form/div[2]/div[3]/div/select")) #get all the options into a list
        optionsList = []
        for item in select.options:
            optionsList.append(item.get_attribute("value"))            
            for optionValue in optionsList:
                select_option(driver, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[3]/div/select', optionValue)

        # Se hace click con los datos llenados
                driver.find_element(By.XPATH, '/html/body/div[11]/div/div/div[2]/form/div[2]/div[4]/button').click()
        
        # Se descargan los datos
                wait = WebDriverWait(driver, 10) 
    #descarga por salas cambia último tr: 1, 2, 3, 4 - no siempre está disponible, el mismo día, las mismas salas
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[1]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[2]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[3]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
                try: 
                    boton_descarga = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[11]/div/div/div[2]/table/tbody/tr[4]/td[1]/a')))
                    boton_descarga.click()
                except: 
                    pass
driver.close


columns = ['Fecha', 'Sala', 'Integrantes', 'Causa']
df = pd.DataFrame(columns = columns)

palabras = ['licencia', 'comisi ón', 'permiso', 'feriado', 'inhabilidad', 'subroga', 'vacancia']


for palabra in palabras:
    print(palabra in text)




pdf_files = glob.glob(os.path.join("C:/Users/jwile/WebScraping/output2",'*.pdf'))

count = 0
extractedtext = ""
for pdf_file in pdf_files:
    print(pdf_file)
    pdfFileObj = open(pdf_file,'rb')               
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)   
    num_pages = pdfReader.numPages
    pdfPageObj = pdfReader.getPage(0)
    text = pdfPageObj.extractText()
    Causa = []
    if re.search("INSTALACI", text):
        Integrantes = re.findall('(?<=ÑOR).*?(?=\s)', text)
        try:
            Fecha = re.findall('(?<=FIRMADIGITAL).*?(?=\\\n)', text)[0]
        except:
            pass
        try:
            Sala = re.findall('(?<=\\s).*?(?=\\sSALA)', text)[0]
        except:
            pass
        for palabra in palabras:         
            if palabra in text:
                Causa.append(palabra)
        df = df.append(
               {"Fecha" : str(Fecha), "Sala" : str(Sala), "Integrantes" : Integrantes, 'Causa' : Causa},
               ignore_index=True,
            )


df['Causa'].head(50)

#Para separar año y día y convertir a número - lleva a formato fecha. Ojo: muy largo y pesado de ejecutar
df['Fecha'] = df['Fecha'].str.replace("treinta y uno", "treintaiuno")
df['Dia'] = df['Fecha'].str.split(' ').str[0] 
df["Ano"] = df["Fecha"].str.extract(r'(\w+)$')
df['Dia'] = df['Dia'].str.replace("treintaiuno", "treinta y uno")

#df['Dia'] = df.Dia.apply(w2n.word_to_num)
df['Dia']

nums = {'^uno$': '1', '^dos$': '2', '^tres$': '3', "^cuatro$":'4', "^cinco$" : '5', "^seis$":'6', "^siete$": '7', 
        '^ocho$':'8',
       '^nueve$':'9', 'diez':'10', 'once':'11', 'doce':'12', 'trece':'13','catorce':'14', 'quince': '15', 'dieciséis':'16',
        'diecisiete':'17', 'dieciocho':'18', 'diecinueve':'19', 'veinte':'20', 'veintiuno':'21','veintidós':'22', 
        'veintitrés':'23', 'veinticuatro':'24', 'veinticinco':'25', 'veintiséis':'26', 'veintisiete':'27',
        'veintiocho':'28', 'veintinueve': '29', '^treinta$': '30', 'treinta y uno': '31'}
        

for old, new in nums.items():
    df['Dia'] = df['Dia'].str.replace(old, new, regex=True)

df['Dia'].value_counts()

nums = {'^veinte$': '20', '^veintiuno$': '21', '^veintidós$': '22'}
        

for old, new in nums.items():
    df['Ano'] = df['Ano'].str.replace(old, new, regex=True)

df['Ano'].value_counts()

#convierte a string para juntar
df['Ano'] = df['Ano'].astype(str)
df['Dia'] = df['Dia'].astype(str)#aisla y convierte a una columna mes
df["Mes"] = df["Fecha"]
df['Mes'] = df['Mes'].str.split(n=1).str[1]
df['Mes'] = df['Mes'].str.lstrip()
df['Mes'] = df['Mes'].str.split(' ').str[1] 

#concatenación fecha en formato
df['Fecha2'] = df['Dia'] + '/' + df['Mes'] + '/' + df['Ano']
#transformación a datetime
df['Fecha2'] = pd.to_datetime(df["Fecha2"], format = '%d/%B/%y')

df2 = df
df2['Presidente'] = df2['Integrantes'].str[0]
df2['Integrante1'] = df2['Integrantes'].str[1]
df2['Integrante2'] = df2['Integrantes'].str[2]
df2['Integrante3'] = df2['Integrantes'].str[3]
df2['Integrante4'] = df2['Integrantes'].str[4]

df3 = df2
df3 = df3.drop(['Fecha', 'Integrantes', 'Dia', 'Ano', 'Mes'], axis = 1)


df3['Causa1'] = df3['Causa'].str[0]
df3['Causa2'] = df3['Causa'].str[1]
df3['Causa3'] = df3['Causa'].str[2]
df3['Causa4'] = df3['Causa'].str[3]


df3 = df3.drop(['Causa'], axis = 1)

df3.drop_duplicates()

df3.to_csv("Segundo intento")

[WDM] - Downloading:  99%|█████████▉| 6.13M/6.21M [00:02<00:00, 1.93MB/s]<ipython-input-1-fab5edface29>:44: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options = chromeOptions)
[WDM] - Downloading: 100%|██████████| 6.21M/6.21M [00:20<00:00, 1.93MB/s]

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[11]/div/div/div[2]/form/div[2]/div[1]/div/select"}
  (Session info: headless chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x011778B3+2193587]
	Ordinal0 [0x01110681+1771137]
	Ordinal0 [0x010241A8+803240]
	Ordinal0 [0x010524A0+992416]
	Ordinal0 [0x0105273B+993083]
	Ordinal0 [0x0107F7C2+1177538]
	Ordinal0 [0x0106D7F4+1103860]
	Ordinal0 [0x0107DAE2+1170146]
	Ordinal0 [0x0106D5C6+1103302]
	Ordinal0 [0x010477E0+948192]
	Ordinal0 [0x010486E6+952038]
	GetHandleVerifier [0x01420CB2+2738370]
	GetHandleVerifier [0x014121B8+2678216]
	GetHandleVerifier [0x012017AA+512954]
	GetHandleVerifier [0x01200856+509030]
	Ordinal0 [0x0111743B+1799227]
	Ordinal0 [0x0111BB68+1817448]
	Ordinal0 [0x0111BC55+1817685]
	Ordinal0 [0x01125230+1856048]
	BaseThreadInitThunk [0x75A26739+25]
	RtlGetFullPathName_UEx [0x771590AF+1215]
	RtlGetFullPathName_UEx [0x7715907D+1165]
